# Testing Classification Models for Nebraska Record Linking
## Import Libraries

In [3]:
import pandas as pd
import re
import sys
sys.path.append(r'R:\JoePriceResearch\Python\Anaconda3\Lib\site-packages')
from jellyfish import jaro_winkler as jw
from jellyfish import soundex as sdx
from jellyfish import damerau_levenshtein_distance as dl
import numpy as np
from sklearn import ensemble
import pickle
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import time
#from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'jellyfish'

## Read in Census Files
The following is the original code used to read in census files. 

In [2]:
c1910 = pd.read_stata(r'R:\JoePriceResearch\record_linking\data\census_1910\data\state_files\Nebraska.dta')
print(c1910.columns)
c1920 = pd.read_stata(r'R:\JoePriceResearch\record_linking\data\census_1920\data\state_files\nebraska.dta')
print(c1920.columns)
cols = [c for c in c1910.columns if c in c1920.columns]
print([c for c in c1910.columns if c not in cols])
print([c for c in c1920.columns if c not in cols])
rncols = [c for c in cols if c not in ['ark1910','ark1920','source_sheet_nbr_ltr','fs_ims_image_id']]
print([c for c in c1910.columns if c not in cols])
print([c for c in c1920.columns if c not in cols])
keep1910 = [r+'1910' for r in rncols]+['ark1910']
keep1920 = [r+'1920' for r in rncols]+['ark1920']
rename1910 = dict(zip(rncols,[x+'1910' for x in rncols]))
rename1920 = dict(zip(rncols,[x+'1920' for x in rncols]))
c1910 = c1910.rename(columns=rename1910)
c1920 = c1920.rename(columns=rename1920)
print([c for c in c1910.columns if c not in keep1910])
print([c for c in c1920.columns if c not in keep1920])
c1910 = c1910[keep1910]
c1920 = c1920[keep1920]
c1910 = c1910.rename(columns={'pr_name_surn1910':'last1910'})
c1920 = c1920.rename(columns={'pr_name_surn1920':'last1920'})
del cols, keep1910, keep1920, rename1910, rename1920, rncols
c1910['first_init1910'] = [x[0] if len(x)>0 else '' for x in c1910['pr_name_gn1910']]
c1920['first_init1920'] = [x[0] if len(x)>0 else '' for x in c1920['pr_name_gn1920']]
c1910['last_init1910'] = [x[0] if len(x)>0 else '' for x in c1910['last1910']]
c1920['last_init1920'] = [x[0] if len(x)>0 else '' for x in c1920['last1920']]
c1910['first_sdx1910'] = [sdx(x.split()[0]) if len(x)>0 else '' for x in c1910['pr_name_gn1910']]
c1920['first_sdx1920'] = [sdx(x.split()[0]) if len(x)>0 else '' for x in c1920['pr_name_gn1920']]
c1910['last_sdx1910'] = [sdx(x) for x in c1910['last1910']]
c1920['last_sdx1920'] = [sdx(x) for x in c1920['last1920']]
c1910['pr_age1910'] = [int(x) if len(x)>0 else np.nan for x in c1910['pr_age1910']]
c1920['pr_age1920'] = [int(x) if len(x)>0 else np.nan for x in c1920['pr_age1920']]
c1910['pr_imm_year1910'] = [int(x) if len(x)>0 else np.nan for x in c1910['pr_imm_year1910']]
c1920['pr_imm_year1920'] = [int(x) if len(x)>0 else np.nan for x in c1920['pr_imm_year1920']]
c1920 = c1920[c1920['pr_age1920']>9]
keep = [bool(1-(x>1911 and x<1921)) for x in c1920['pr_imm_year1920']]
c1920 = c1920[keep]
del keep

Index(['ark1910', 'event_place', 'event_township', 'fs_ims_image_id', 'pr_age',
       'pr_bir_place', 'pr_bir_year_est', 'pr_ethnicity_css',
       'pr_fthr_bir_place', 'pr_imm_year', 'pr_marital_status',
       'pr_mthr_bir_place', 'pr_name_gn', 'pr_name_surn', 'pr_race_or_color',
       'pr_relationship_code', 'pr_sex_code', 'source_household_id',
       'source_sheet_nbr_ltr'],
      dtype='object')
Index(['ark1920', 'event_country', 'event_county', 'event_date',
       'event_district', 'event_place', 'event_place_level_1',
       'event_place_level_1_type', 'event_place_level_2',
       'event_place_level_2_orig', 'event_place_level_2_type',
       'event_place_level_3', 'event_place_level_3_orig',
       'event_place_level_3_type', 'event_place_level_4',
       'event_place_level_4_type', 'event_place_orig', 'event_state',
       'event_township', 'event_type', 'event_year', 'ext_2_person_id',
       'ext_film_nbr', 'ext_pub_nbr', 'ext_repository_name', 'fs_batch_id',
       'fs

## Create Bins, Features, Merge True Matches, Identify False Pairings
I used Isaac's origninal method for creating bins and putting the data together. Currently I'm working on binning based on results of our probit model, but I'm still figuring out how to get that to work efficiently.

In [3]:
c1910['bin'] = [x[0]+x[1]+x[2]+x[3]+x[4]+x[5]+x[6] for x in zip(c1910['first_sdx1910'],c1910['last_sdx1910'],c1910['pr_bir_place1910'],c1910['pr_race_or_color1910'],c1910['pr_sex_code1910'],c1910['pr_mthr_bir_place1910'],c1910['pr_fthr_bir_place1910'])]
c1920['bin'] = [x[0]+x[1]+x[2]+x[3]+x[4]+x[5]+x[6] for x in zip(c1920['first_sdx1920'],c1920['last_sdx1920'],c1920['pr_bir_place1920'],c1920['pr_race_or_color1920'],c1920['pr_sex_code1920'],c1920['pr_mthr_bir_place1920'],c1920['pr_fthr_bir_place1920'])]
bins = list(c1910['bin'])+list(c1920['bin'])
bins = list(set(bins))
bindict = dict(zip(bins, range(len(bins))))
c1910['bin'] = c1910['bin'].apply(lambda x:bindict[x])
c1920['bin'] = c1920['bin'].apply(lambda x:bindict[x])
del bins, bindict

In [4]:
df = pd.merge(c1910, c1920, on='bin', how='inner')
#del c1910_1, c1920_1
df['age_diff'] = [abs(x[1]-x[0]-10) for x in zip(df['pr_age1910'],df['pr_age1920'])]
df = df[df['age_diff']<4]

In [5]:
df['county1910'] = [x.split(', ')[-3] if x.count(', ')>1 else '' for x in df['event_place1910']]
df['county1920'] = [x.split(', ')[-3] if x.count(', ')>1 else '' for x in df['event_place1920']]
df['township1910'] = [re.sub('^ +| +$| ?[0-9]+| ?Ward| ?Precinct', '', t) for t in df['event_township1910']]
df = df.drop('event_township1910', axis=1)
df['township1920'] = [re.sub('^ +| +$| ?[0-9]+| ?Ward| ?Precinct', '', t) for t in df['event_township1920']]
df = df.drop('event_township1920', axis=1)
df['first1910'] = [x.split()[0] if len(x)>0 else '' for x in df['pr_name_gn1910']]
df['first1920'] = [x.split()[0] if len(x)>0 else '' for x in df['pr_name_gn1920']]
df['second1910'] = [x.split()[1] if ' ' in x else '' for x in df['pr_name_gn1910']]
df['second1920'] = [x.split()[1] if ' ' in x else '' for x in df['pr_name_gn1920']]
df['mid_init1910'] = [x[0] if len(x)>0 else '' for x in df['second1910']]
df['mid_init1920'] = [x[0] if len(x)>0 else '' for x in df['second1920']]
df['second_sdx1910'] = [sdx(x) for x in df['second1910']]
df['second_sdx1920'] = [sdx(x) for x in df['second1920']]

df['first_match'] = [x[0]==x[1] for x in zip(df['first1910'],df['first1920'])]
df['last_match'] = [x[0]==x[1] for x in zip(df['last1910'],df['last1920'])]
df['f1_match'] = df['first_init1910']==df['first_init1920']
df['m1_match'] = df['mid_init1910']==df['mid_init1920']
df['l1_match'] = df['last_init1910']==df['last_init1920']
df['f1_noconflict'] = [x[0]==x[1] or len(x[0])==0 or len(x[1])==0 for x in zip(df['first_init1910'],df['first_init1920'])]
df['m1_noconflict'] = [x[0]==x[1] or len(x[0])==0 or len(x[1])==0 for x in zip(df['mid_init1910'],df['mid_init1920'])]
df['l1_noconflict'] = [x[0]==x[1] or len(x[0])==0 or len(x[1])==0 for x in zip(df['last_init1910'],df['last_init1920'])]
df['first_sdx_match'] = [x[0]==x[1] for x in zip(df['first_sdx1910'],df['first_sdx1920'])]
df['last_sdx_match'] = [x[0]==x[1] for x in zip(df['last_sdx1910'],df['last_sdx1920'])]
df['first_jw'] = [jw(x[0],x[1]) for x in zip(df['first1910'],df['first1920'])]
df['last_jw'] = [jw(x[0],x[1]) for x in zip(df['last1910'],df['last1920'])]
df['mid_jw'] = [jw(x[0],x[1]) for x in zip(df['second1910'],df['second1920'])]
#df['pr_age1910'] = [int(x) if len(x)>0 else np.nan for x in df['pr_age1910']]
#df['pr_age1920'] = [int(x) if len(x)>0 else np.nan for x in df['pr_age1920']]
df['age_pm1'] = [abs(x[0]-x[1]) in range(9,12) for x in zip(df['pr_age1910'],df['pr_age1920'])]
df['age_pm2'] = [abs(x[0]-x[1]) in range(8,13) for x in zip(df['pr_age1910'],df['pr_age1920'])]
df['age_pm3'] = [abs(x[0]-x[1]) in range(7,14) for x in zip(df['pr_age1910'],df['pr_age1920'])]
df['mbp_match'] = df['pr_mthr_bir_place1910']==df['pr_mthr_bir_place1920']
df['fbp_match'] = df['pr_fthr_bir_place1910']==df['pr_fthr_bir_place1920']
df['race_match'] = df['pr_race_or_color1910']==df['pr_race_or_color1920']
df['ethn_match'] = df['pr_ethnicity_css1910']==df['pr_ethnicity_css1920']
df['sex_match'] = df['pr_sex_code1910']==df['pr_sex_code1920']
df['town_match'] = df['township1910']==df['township1920']
df['county_match'] = df['county1910']==df['county1920']
df['first_xjw12'] = [jw(x[0],x[1]) for x in zip(df['first1910'],df['second1920'])]
df['first_xjw21'] = [jw(x[0],x[1]) for x in zip(df['second1910'],df['first1920'])]
df['first_xsdx12'] = [x[0]==x[1] for x in zip(df['first_sdx1910'],df['second_sdx1920'])]
df['first_xsdx21'] = [x[0]==x[1] for x in zip(df['second_sdx1910'],df['first_sdx1920'])]
df['first_maxjw'] = [max(x) for x in zip(df['first_jw'],df['first_xjw12'],df['first_xjw21'])]
df['first_maxsdx'] = [max(x) for x in zip(df['first_sdx_match'],df['first_xsdx12'],df['first_xsdx21'])]
df['min_jw'] = [min(x) for x in zip(df['first_maxjw'],df['last_jw'])]
df['imm_match'] = df['pr_imm_year1910']==df['pr_imm_year1920']

features = ['first_match','last_match','f1_match','m1_match','l1_match','f1_noconflict',
            'm1_noconflict','l1_noconflict','first_sdx_match','last_sdx_match',
            'first_jw','last_jw','mid_jw','age_diff','age_pm1',
            'age_pm2','age_pm3','mbp_match','fbp_match','race_match','ethn_match',
            'sex_match','town_match','county_match','first_maxjw','first_maxsdx',
            'min_jw','imm_match']
df['total'] = [np.sum([i==1 for i in x]) for x in list(zip(*[list(df[c]) for c in df.columns]))]
features += ['total']

In [6]:
fs = pd.read_stata(r'R:\JoePriceResearch\record_linking\data\crosswalks\arkhints1910_1920.dta')
arks1 = set(df['ark1910'])
arks2 = set(df['ark1920'])
fs = [x for x in zip(fs['ark1910'],fs['ark1920']) if x[0] in arks1]
fs = [x for x in fs if x[1] in arks2]
fs = pd.DataFrame(fs)
fs['true'] = [1]*fs.shape[0]
fs = fs.rename(columns={0:'ark1910',1:'ark1920'})
df = pd.merge(df, fs, on=['ark1910','ark1920'], how='left')
df['true'] = [x==1 for x in df['true']]

In [7]:
group = df[['ark1920','true']].groupby(df['ark1920'])
train = dict(group['true'].max())
df['train2'] = df['ark1920'].apply(lambda x:train[x])
group = df[['ark1910','true']].groupby(df['ark1910'])
train = dict(group['true'].max())
df['train1'] = df['ark1910'].apply(lambda x:train[x])
train = df[df['train1']+df['train2']>0].drop(['train1','train2'], axis=1)
pred = df[df['train1']+df['train2']==0].drop(['train1','train2'], axis=1) 
train = train[['true','ark1910','ark1920']+features]   
pred = pred[['ark1910','ark1920']+features]   
del df

C:\Users\marsdena\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:180: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


## Putting our Models Together:
### First I perform a train-test split on the training data. 

In [8]:
seed = 21
np.random.seed(seed)
train = train.drop(['ark1910','ark1920'],axis=1)
y = train['true']
X = train.drop(['true'],1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3)


### Gradient Boosting
The timeit is used to measure the processing time of each model. This model here is gradient boosting. I used the one imported from sklearn's ensemble package. 

In [9]:
import timeit
tic = timeit.default_timer()
model = ensemble.GradientBoostingClassifier(n_estimators = 1000, learning_rate = 0.1, max_depth = 6, max_features = 0.8, random_state = 5)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
toc = timeit.default_timer()

time = toc - tic
print(time)

315.7601168869999


#### Accuracy Measures:
First I created a confusion matrix to caclulate precision and recall. I do this again with the classification_report tool from sklearn which makes it easier to get these values. 

In [10]:
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
predictions = cross_val_predict(model, X_test, y_test, cv=3)
confusion_matrix(y_test, predictions)

array([[16674,  4768],
       [ 2293, 65287]], dtype=int64)

In [11]:
from sklearn.metrics import confusion_matrix, classification_report
print(confusion_matrix(y_test, predictions))
predictions = model.predict(X_test)
print(classification_report(y_test,predictions))

[[16674  4768]
 [ 2293 65287]]
             precision    recall  f1-score   support

      False       0.87      0.80      0.84     21442
       True       0.94      0.96      0.95     67580

avg / total       0.92      0.92      0.92     89022



The following is another measure of the model's score. 

In [12]:
in_rf_acc = model.score(X_train, y_train)
print('In sample predictions: ' + str(in_rf_acc))

In sample predictions: 0.9322773993211853


### Random Forests
Essentially the same as Gradient Boosting but with Random Forests

In [13]:
tic = timeit.default_timer()
model = ensemble.RandomForestClassifier(n_estimators = 1000, max_features = 0.8, max_depth = 6, n_jobs = -1)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
toc = timeit.default_timer()

time = toc - tic
print(time)

86.89002848399991


In [19]:
predictions = cross_val_predict(model, X_test, y_test, cv=3)
confusion_matrix(y_test, predictions)

array([[15788,  5654],
       [ 1403, 66177]], dtype=int64)

In [20]:
print(confusion_matrix(y_test, predictions))
predictions = model.predict(X_test)
print(classification_report(y_test,predictions))

[[15788  5654]
 [ 1403 66177]]
             precision    recall  f1-score   support

      False       0.83      0.84      0.83     21442
       True       0.95      0.95      0.95     67580

avg / total       0.92      0.92      0.92     89022



In [16]:
in_rf_acc = model.score(X_train, y_train)
print('In sample predictions: ' + str(in_rf_acc))

In sample predictions: 0.9215319066990829


### Logit

In [1]:
from sklearn.linear_model import LogisticRegression

In [2]:
tic = timeit.default_timer()
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
toc = timeit.default_timer()

time = toc - tic
print(time)

NameError: name 'timeit' is not defined

In [ ]:
predictions = cross_val_predict(model, X_test, y_test, cv=3)
confusion_matrix(y_test, predictions)

In [ ]:
print(confusion_matrix(y_test, predictions))
predictions = model.predict(X_test)
print(classification_report(y_test,predictions))

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
tic = timeit.default_timer()
model = MLPClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
toc = timeit.default_timer()

time = toc - tic
print(time)

In [ ]:
predictions = cross_val_predict(model, X_test, y_test, cv=3)
confusion_matrix(y_test, predictions)

In [ ]:
print(confusion_matrix(y_test, predictions))
predictions = model.predict(X_test)
print(classification_report(y_test,predictions))

### XGBOOST
Here is the model I most recently used for the XGBClassifier. I can't run it on here because it won't let me download the library on any school computer. So I've been saving my training data and running it on my personal. To test it, I used the same commands as before with random forests and gradient boosting. 

In [ ]:
from xgboost import XGBClassifier 
tic = timeit.default_timer()
model = XGBClassifier(max_depth=6, learning_rate = 0.1, n_estimators = 1000)
model.fit(X_train, y_train)
predictions = model.predict(X_test)
toc = timeit.default_timer()

time = toc - tic
print(time)